In [1]:
import pandas as pd
import conn_way
from datetime import timedelta

通话记录的数据截止到7月底，对应合同数

In [2]:
l1 = pd.read_csv('./call_data_source/call_data_1711to1805.csv',dtype={'CALL_NUMBER':'str'}).LOAN_ID.value_counts().index.tolist()

In [3]:
l2 = pd.read_csv('./call_data_source/call_data_1805to1806.csv',dtype={'CALL_NUMBER':'str'}).LOAN_ID.value_counts().index.tolist()

In [4]:
l3 = pd.read_csv('./call_data_source/call_data_1806to1807.csv',dtype={'CALL_NUMBER':'str'}).LOAN_ID.value_counts().index.tolist()

In [5]:
call_loan_id = list(set(l1).union(set(l2)).union(set(l3)))

In [6]:
len(call_loan_id)

11956

更新label后，当前有label并且包含基本信息的合同数

In [7]:
conn = conn_way.credit()

In [8]:
applydate = pd.read_sql('SELECT LOAN_ID,APPLY_DATE FROM HS_APPLY',conn)

In [9]:
label = pd.read_csv('./label-2019-03.csv').rename(columns={'loan_id':'LOAN_ID'})

In [10]:
label = pd.merge(applydate,label,on='LOAN_ID',how='inner')

In [11]:
label.drop_duplicates('LOAN_ID',inplace=True)

In [12]:
label.shape

(62426, 7)

In [13]:
label.classification.value_counts()

good      34047
bad       25666
unknow     2713
Name: classification, dtype: int64

以summary表来统计征信数据，当前备份库中征信数据合同数

In [14]:
def updatesummary():

    conn = conn_way.hsdc()
    sql = r'SELECT b.ID_CARD,b.SELECTTIME,s.* FROM CREDIT_BASIC b INNER JOIN CREDIT_SUMMARY s ON b.UUID = s.UUID'
    df_summary = pd.read_sql(sql, conn)
    conn.close()

    conn = conn_way.credit()
    sql = r'SELECT loan_id,id_number AS id_card,apply_date FROM HS_APPLY'
    df_apply = pd.read_sql(sql, conn)
    conn.close()

    df_summary.SELECTTIME = df_summary.SELECTTIME.astype('datetime64[ns]')
    df = pd.merge(df_apply, df_summary, on='ID_CARD', how='inner')
    df['M'] = df.APPLY_DATE - df.SELECTTIME
    df = df[(df.M > timedelta(days=0)) & (df.M < timedelta(days=10))]
    """apply表可能出现客户反复申请的情况，对应的都是同一份征信，将会导致作为主键的id重复，需要做唯一值处理"""
    df.drop_duplicates('LOAN_ID',inplace=True,keep='last')

    return df

In [15]:
df = updatesummary()

In [16]:
df.columns = df.columns.map(lambda x: x.lower())

In [17]:
df.drop(['id','order_id','check_result','update_time_ini','uuid','m'],axis=1,inplace=True)

In [18]:
credit_loan_id = df.loan_id.tolist()

In [19]:
conn =conn_way.data_operation()
localhost_credit_list = pd.read_sql('SELECT loan_id FROM credit_summary',conn).loan_id.tolist()

In [20]:
credit_loan_id = list(set(credit_loan_id).union(set(localhost_credit_list)))

In [21]:
len(credit_loan_id)

130094

包含客户基本信息、征信数据、并且有label的数据条目

In [22]:
label_credit = label[label.LOAN_ID.isin(credit_loan_id)]

In [23]:
label_credit.shape

(42965, 7)

In [24]:
label_credit.classification.value_counts()

good      24833
bad       15446
unknow     2686
Name: classification, dtype: int64

In [ ]:
23929 + 13894

包含客户基本信息、征信数据、通话记录、并且有label的数据条目

In [25]:
label_credit_call = label_credit[label_credit.LOAN_ID.isin(call_loan_id)]

In [26]:
label_credit_call.shape

(11951, 7)

In [27]:
label_credit_call.classification.value_counts()

good      9161
bad       2338
unknow     452
Name: classification, dtype: int64

包含客户基本信息、通话记录、并且有label的数据条目

In [28]:
label_call = label[label.LOAN_ID.isin(call_loan_id)]

In [29]:
label_call.shape

(11951, 7)

In [30]:
label_call.classification.value_counts()

good      9161
bad       2338
unknow     452
Name: classification, dtype: int64

In [39]:
label_call[['LOAN_ID','APPLY_DATE','con_no','classification']].to_csv('./result.csv', index=False)